In [1]:
import pandas as pd
from pandas import DataFrame
import os
import sys
import csv
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
plt.show()
import glob
import seaborn as sns


outdir='/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/aprA/'
if os.path.isdir(outdir):
    print(outdir, "already exists.")
else:
    os.mkdir(outdir)
    print(outdir, "is created.")

os.chdir('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/aprA/')    

images='/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_images/aprA'
if os.path.isdir(images):
    print(images, "already exists.")
else:
    os.mkdir(images)
    print(images, "is created.")
    
matplotlib.__version__

/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/aprA/ already exists.
/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_images/aprA already exists.


'3.3.3'

rerun on 2021-12-07 after correcting for salinity of media samples

In [2]:
!module use /mod/scgc
!module load seqtk/1.0.75




In [3]:
! grep "Adenylylsulfate" /mnt/scgc/simon/microg2p/Data/SAG_genes/all_GoM_genes.ffn > apr_genes.txt
! grep "alpha" apr_genes.txt > aprA_genes.txt
alpha=pd.read_csv('aprA_genes.txt', sep=" ", header=None, usecols=[0])
alpha[0]=alpha[0].str[1:]
alist=list(alpha[0])

file=open('aprA_genes.txt','w')
for items in alist:
    file.writelines(items+'\n')
file.close()


! grep "beta" apr_genes.txt > aprB_genes.txt
beta=pd.read_csv('aprB_genes.txt', sep=" ", header=None, usecols=[0])
beta[0]=beta[0].str[1:]
blist=list(beta[0])

file=open('aprB_genes.txt','w')
for items in blist:
    file.writelines(items+'\n')
file.close()

! head aprA_genes.txt

AH-135-A04_00020
AH-135-A07_00715
AH-135-B20_00288
AH-135-C17_01258
AH-135-D17_00233
AH-135-F23_00368
AH-135-G03_00614
AH-135-J02_00431
AH-135-K06_00420
AH-135-K20_00503


In [4]:
!seqkit grep -f aprA_genes.txt /mnt/scgc/simon/microg2p/Data/SAG_genes/all_GoM_genes.ffn  -o aprA_genes.fasta
!seqkit grep -f aprB_genes.txt /mnt/scgc/simon/microg2p/Data/SAG_genes/all_GoM_genes.ffn  -o aprB_genes.fasta


! head aprA_genes.fasta

>AH-135-A04_00020 Adenylylsulfate reductase subunit alpha
ATGGCTAAGAGAAAAACTGTTTGGGAAGACTGTGATATATTAGTTGTTGGTGGTGGTATG
GCTGGAACTGGTGCTTCTTATGAGGCTAGATATTGGGGCCGTGATATGAGAATTATATGT
GCCGAAAAGGCAAACATTGATCGAAGCGGTGCAGTAGCTCAAGGCCTTTATGCGATCAAC
TGCTATATGGGAATGCAGTGGGATGAAAACCAACCCGAAGACCATGTTCGATATGCACGT
AATGACTTGATGGGTTTAGTTCGTGAGGACCTTGCTTTTGATATGGCTCGACATGTTGAC
TCTGCCGTTCACAAATTTGAAGAATGGGGCCTACCCATTATGAGAAATGAAGAGACTGGT
CGTTACCAGCGCGAAGGTAAATGGCAAATAATGATTCACGGTGAAAGCTACAAACCTATC
GTTGCAGAAGCTGCTCGCAAATCAGTAGACAAAATTTATAACCGAATTATGATTACGCAT
CTTTTAATGGATGAGGCCAAGGAGAATAGAGTTGGTGGAGCAGTTGGCTTTAATTGCCGT


In [5]:
! seqkit translate aprA_genes.fasta -o aprA_genes.faa --trim
! seqkit translate aprB_genes.fasta -o aprB_genes.faa --trim
! head aprB_genes.faa

>AH-135-A04_00019 Adenylylsulfate reductase subunit beta
MPTFVYMTRCDGCGHCVDICPSDIMHIDKTIRRAVNIEPNFCWECYSCVKACPQNAIDVR
GYADFAPMGHSVRVLREEEKGTISWKIKFRDGREKNFVSPITTQPWGEKIPKLADLEVPD
QAALDSQLLCFEPDALNIETGLPVINKDKLKEGVYY
>AH-135-A07_00716 Adenylylsulfate reductase subunit beta
MSTFVYMTRCDGCGHCVDICPSDIMHIDENIRRAVNIEPNFCWECYSCVKACPNHAIDVR
GYADFAPMGHSVRVRREEEKGTISWKIKFRNGTTKDFVSPITTKPWGKFIPKLAEGEKPN
QGEINSQMLYTEPNGLNIDGKLPSVPKDTFKKGVYY
>AH-135-B20_00287 Adenylylsulfate reductase subunit beta
MPTFVYMTRCDGCGHCVDICPSDIMHIDKVNRRAYNIEPTHCWECFSCVKACPQNAIDVR


In [6]:
SAG_data=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/Summary_files/All_GoM_SAGs_1cell_20kb_decon_531normalized_predresp_rate_GTDBclass.csv', 
                     index_col=0, usecols=['name', 'GTDB_classification'])
SAG_data
alpha[0]=alpha[0].str[:10]
alpha=alpha.merge(SAG_data, how='left', left_on=0, right_on='name')
alpha=alpha.rename(columns={0:'SAG'})

beta[0]=beta[0].str[:10]
beta=beta.merge(SAG_data, how='left', left_on=0, right_on='name')
beta=beta.rename(columns={0:"SAG"})

alpha

,SAG,GTDB_classification
0,AH-135-A04,Thioglobus
1,AH-135-A07,Unclassified
2,AH-135-B20,ASP10-02a
3,AH-135-C17,Pelagibacter
4,AH-135-D17,Thioglobus
...,...,...
449,AH-709-P06,Pelagibacter
450,AH-709-P07,Pelagibacter
451,AH-709-P10,Pelagibacter
452,AH-709-P18,Pelagibacter


In [7]:
#check the length of the genes and get rid of any that are <75% of full length
!seqkit stats aprA_genes.faa
!seqkit stats aprB_genes.faa

file            format  type     num_seqs  sum_len  min_len  avg_len  max_len
aprA_genes.faa  FASTA   Protein       454  273,328       60      602      776
file            format  type     num_seqs  sum_len  min_len  avg_len  max_len
aprB_genes.faa  FASTA   Protein       438   68,377       75    156.1      166


In [8]:
# remove any sequences that are <75% of the average length 
!seqkit seq -m 450 aprA_genes.faa > aprA_genes_75perc.faa
!seqkit seq -m 120 aprB_genes.faa > aprB_genes_75perc.faa

! cat aprA_genes_75perc.faa aprA_ref_genes.faa > aprA_genes_75perc_and_ref.faa

!seqkit stats aprA_genes_75perc.faa
!seqkit stats aprB_genes_75perc.faa

[WARN] you may switch on flag -g/--remove-gaps to remove spaces
[WARN] you may switch on flag -g/--remove-gaps to remove spaces
cat: aprA_ref_genes.faa: No such file or directory
file                   format  type     num_seqs  sum_len  min_len  avg_len  max_len
aprA_genes_75perc.faa  FASTA   Protein       437  269,488      463    616.7      776
file                   format  type     num_seqs  sum_len  min_len  avg_len  max_len
aprB_genes_75perc.faa  FASTA   Protein       437   68,302      120    156.3      166


In [9]:
! module load muscle/3.8.31
! muscle -in aprA_genes_75perc_and_ref.faa.faa -out aprA_genes_75perc_and_ref_muscle_align.faa
! muscle -in aprB_genes_75perc.faa -out aprB_75perc_muscle_align.faa



MUSCLE v3.8.1551 by Robert C. Edgar

http://www.drive5.com/muscle
This software is donated to the public domain.
Please cite: Edgar, R.C. Nucleic Acids Res 32(5), 1792-97.


*** ERROR ***  Cannot open 'aprA_genes_75perc_and_ref.faa.faa' errno=2


MUSCLE v3.8.1551 by Robert C. Edgar

http://www.drive5.com/muscle
This software is donated to the public domain.
Please cite: Edgar, R.C. Nucleic Acids Res 32(5), 1792-97.

aprB_genes_75perc 437 seqs, lengths min 120, max 166, avg 156
00:00:01    21 MB(-4%)  Iter   1  100.00%  K-mer dist pass 1
00:00:01    21 MB(-4%)  Iter   1  100.00%  K-mer dist pass 2
00:00:01   65 MB(-12%)  Iter   1  100.00%  Align node       
00:00:01   66 MB(-12%)  Iter   1  100.00%  Root alignment
00:00:01   66 MB(-12%)  Iter   2  100.00%  Refine tree   
00:00:01   66 MB(-12%)  Iter   2  100.00%  Root alignmentent
00:00:01   66 MB(-12%)  Iter   2  100.00%  Root alignment
00:00:03   66 MB(-12%)  Iter   3  100.00%  Refine biparts
00:00:04   66 MB(-12%)  Iter   4  100.00%